In [ ]:
# Packages
!pip install numpy
!pip install sympy
!pip install networkx
!pip install plotly
!pip install ipython
!pip install pylatex
!pip install -U kaleido
# Import math stuff
import numpy as np
from sympy import *
from decimal import Decimal
# Import random
import random
from random import randrange
# To create a Latex document 
from pylatex import Document, Section, Subsection, Command, Alignat, NoEscape, Math, TikZ, Figure, Axis, Tabular
from pylatex.utils import italic, NoEscape
import os
# Import packages for graphics
import networkx as nx
import plotly.offline as py
import plotly.graph_objects as go

In [ ]:
# Constructs a function d from B to N^2 so that each node in N is 'in' the image of d (as a starting node or ending node).
# The i-th entry of d is the image of the i-th branch in B
def functiond(N,B):
  d=[(N[0],N[1])] # Without loss of generality, the branch 0 goes from the node 0 to the node 1
  Nodesind={N[0],N[1]} # Nodes in d.
  for i in range(1,len(B)):
    if (i+1 <= len(N)-1) and (N[i+1] not in Nodesind):
      # If the i-th node isn't 'in' the image of d, we now force it to be in it.
      node=N[i+1]
      endnodes=[node,random.choice(N)] 
      while len(set(endnodes))==1:
        endnodes=[node,random.choice(N)]
      direction=random.choice([[1,0],[0,1]])
      starting_node=endnodes[direction[0]]
      ending_node=endnodes[direction[1]]
    else:
      starting_node=random.choice(N)
      ending_node=random.choice(N)
      while starting_node==ending_node:
        ending_node=random.choice(N)
    d.append((starting_node,ending_node))
    Nodesind.update({starting_node,ending_node})
  d=tuple(d)
  return d

In [ ]:
# Generates a one-dimensional complex G=(N,B,d) with n nodes and b branches. The only conditions: b>=n-1, and n>1.
# For simplicity, these tuples are of the form B=(0,1,...,b-1) and N=(0,1,...,n-1)
def complex(n,b):
  N=tuple(range(n))
  B=tuple(range(b))
  d=functiond(N,B)
  return(N,B,d)

In [ ]:
# Generates a maximal forest T=(Nt,Bt,dt) for a complex G=(N,B,d)
def maxforest(N,B,d):
  # The nodes, branches and the function dt of T are in construction: 
  Nt={ 0 }
  Bt=[ ]
  dt=[[ ]]*len(B)
  Neigh=[{ }]*len(N) # The i-th entry will contain the node i and its neighbourhoods via a branch in Bt
  for node in N:
    Neigh[node]={node}
  for branch in B:
    starting_node=d[branch][0]
    ending_node=d[branch][1]
    # We want to know if branch forms a mesh with branches in Bt
    # The question is: Is ending_node connected to starting_node via Bt?
    answer='unknown'
    if (starting_node not in Nt) or (ending_node not in Nt): 
      answer='no'
    else:
      # It is possible that 'branch' is contained in a mesh in T
      Connected=Neigh[starting_node] # We will calculate some of the nodes connected to starting_node via branches in Bt
      while answer=='unknown':
        if ending_node in Connected:
          answer='yes'
        else:
          MoreConn=Connected # Trying to find more connected nodes
          for node in Connected:
            MoreConn=MoreConn.union(Neigh[node])
          if MoreConn==Connected:
            answer='no'
          else:
            Connected=MoreConn # Actualizing what we know
    if answer=='no':
      # There isn't a mesh containing 'branch' and the branches in Bt, so 'branch' and its end-nodes belong to T
      Nt.update({starting_node,ending_node})
      Bt.append(branch)
      dt[branch]=d[branch]
      Neigh[starting_node].add(ending_node)
      Neigh[ending_node].add(starting_node)
  Nt=tuple(Nt)
  Bt=tuple(Bt)
  dt=tuple(dt)
  return (Nt,Bt,dt)

In [ ]:
# Finds a mesh containing bra and branches of the forest T=(Nt,Bt,dt)
def findmesh(bra,N,Bt,d):
  sta_node=d[bra][0]
  end_node=d[bra][1]
  Neigh=[{ }]*len(N) # The i-th entry will contain the node i and its neighbourhoods via a branch in Bt (of less value than bra)
  for node in N:
    Neigh[node]={node}
  for branch in Bt:
    if branch<bra:
      starting=d[branch][0]
      ending=d[branch][1]
      Neigh[starting].add(ending)
      Neigh[ending].add(starting)
  Separated=[{ }]*(len(Bt)+1) # The i-th entry will contain the nodes separated to sta_node by a distance of i branches
  Separated[0]={sta_node}
  Separated[1]=Neigh[sta_node].difference({sta_node})
  Known=Neigh[sta_node] # All known nodes connected to starting_node via branches in B
  count=1
  while end_node not in Known:
    Separated[count+1]={sta_node}
    for node in Separated[count]:
      Separated[count+1].update(Neigh[node])
    Separated[count+1].difference_update(Known)
    Known.update(Separated[count+1])
    count=count+1
  if { } in Separated:
    summands=Separated.index({ })
  else:
    summands=len(Bt)+1
  mesh=[[bra,1]] # The i-th element will contain the i-th branch of the mesh and its sign
  arrow=end_node # We use this to construct a path
  for i in range(1,summands):
    PosNewNodes=Separated[summands-i-1] # Possible new nodes the path can meet
    index=0
    endnodes={d[index][0],d[index][1]}
    while (index not in Bt) or (arrow not in endnodes) or (endnodes.isdisjoint(PosNewNodes)):
      index=index+1
      endnodes={d[index][0],d[index][1]}
    if d[index][0]==arrow:
      mesh.append([index,1])
      arrow=d[index][1]
    else:
      mesh.append([index,-1])
      arrow=d[index][0]
  return mesh

In [ ]:
# Constructs \sigma function
def sigmafu(N,B,d,Bt):
  numbran=len(B)-len(Bt) # Number of branches in B\Bt
  if numbran==0:
    Sigma=[[0]]*len(B)
  else:
    Sigma=[[0]*numbran]*len(B)
  sigma=np.array(Sigma)
  i=0 
  for branch in B:
    if branch not in Bt:
      mesh=findmesh(branch,N,Bt,d)
      for j in range(0,len(mesh)):
        sigma[mesh[j][0]][i]=mesh[j][1]
      i=i+1
  sigma=Matrix(sigma)
  return sigma

In [ ]:
# Generates randomly the properties of the resistors
def resistor(b):
  Kp=[[0]*1]*b
  Wp=[[0]*1]*b
  Rp=[[0]*b]*b
  K=np.array(Kp)
  W=np.array(Wp)
  R=np.array(Rp)
  for i in range(0,b):
    K[i][0]=random.choice(range(-5, 6))
    W[i][0]=random.choice(range(-5, 6))
    R[i][i]=random.choice([-4,-3,-2,-1,0,0,0,2,3,4])
  K=Matrix(K)
  W=Matrix(W)
  R=Matrix(R)
  return (K,W,R)

In [ ]:
# Generates randomly the properties of the resistive elements (positive resistance)
def resistive(b):
  Kp=[[0]*1]*b
  Wp=[[0]*1]*b
  Rp=[[0]*b]*b
  K=np.array(Kp)
  W=np.array(Wp)
  R=np.array(Rp)
  for i in range(0,b):
    K[i][0]=random.choice(range(-10, 11))
    W[i][0]=random.choice(range(-10, 11))
    R[i][i]=random.choice(range(1,16))
  K=Matrix(K)
  W=Matrix(W)
  R=Matrix(R)
  return (K,W,R)

In [ ]:
# Determines if sRsigma is invertible and finds its inverse
def isinvert(R,sigma):
  s=transpose(sigma)
  sRsigma=s*(R*sigma)
  if det(sRsigma)==0:
    invsRs=0
    answer=0
  else:
    invsRs=sRsigma.inv()
    answer=1
  return (sRsigma,invsRs,answer)

In [ ]:
# Finds solutions for V y I in terms of J, W y R and sigma
def MeshCurrentMethod(K,W,R,sigma,invsRs):
  s=transpose(sigma)
  J=invsRs*(s*(R*K-W))
  I=sigma*J
  V=R*(I-K)+W
  return (I,V)

In [ ]:
# Finds a hypothetical electrostatic potential function, given N, B, d and V
# V=[[voltage in 1], [voltage in 2], ..., [voltage in b]] is a column vector
def potential(N,B,d,V):
  V=np.array(V)
  n=len(N)
  # We'll define as many ground nodes (with zero potential) as connected components, we start with the 0-th node
  P=[{ }]*n 
  P[0]=0
  definednodes=[0] 
  # Have we defined all the nodes, i. e., P[node]!={ } for every node in N?
  answer='no'
  iterations=0
  while answer=='no':
    for branch in B:
      sta_node=d[branch][0]
      end_node=d[branch][1]
      if (sta_node in definednodes) and (end_node not in definednodes):
        P[end_node]=P[sta_node]-V[branch][0]
        definednodes.append(end_node)
      else: 
        if (sta_node not in definednodes) and (end_node in definednodes):
          P[sta_node]=P[end_node]+V[branch][0]
          definednodes.append(sta_node)
    # If the number of iterations is greater than the number of branches and answer is 'no', 
    # there are more connected components, so we have to ground another node
    iterations=iterations+1 
    if len(definednodes)==n:
      answer='yes'
    if iterations>len(B):
      undefinednodes=list(set(N).difference(set(definednodes)))
      rand=random.choice(undefinednodes)
      P[rand]=0
      definednodes.append(rand)
      iterations=0
  Pvector=[]
  for i in range(0,len(P)):
    Pvector.append([P[i]])
  return P, Pvector

In [ ]:
# Proves or refutes that P is a potential function that generates V
def isitapotencial(N,B,d,V,P):
  V=np.array(V)
  answer='yes'
  for branch in B:
    sta_node=d[branch][0]
    end_node=d[branch][1]
    if V[branch][0]!=P[sta_node]-P[end_node]:
      answer='no'
  return answer

In [ ]:
# All the ingredients to generate a complex, simulate its resistors, and find their currents and voltages are already defined. 
# The following functions are just to improve the ploting and to make graphics of the complexes

# To express a string number as an integer or with a certain amount of numbers before the decimal point
def express(number):
  num=str(number)
  reso=4 # Resolution
  if num.count('/')==0 and num.count('.')==0:
    return num
  else:
    rational=float(number)
    approx=round(rational,reso)
    num=str(approx)
    return num


def expressrational(number):
  num=str(number)
  reso=4 # Resolution
  if num.count('/')==0 and num.count('.')==0:
    return num
  else:
    slashpos=num.find('/')
    numerator=num[0:slashpos]
    denominator=num[slashpos+1:]
    num=str(r"\frac{"+numerator+"}{"+denominator+"}")
    return num


# Prints a set as in LaTeX
def latexset(Set):
  text='{'
  for i in range(0,len(Set)-1):
    element=express(Set[i])
    text=text+element+', '
  element=express(Set[len(Set)-1])
  text=text+element+'}'
  return text


# Prints a tuple
def latextuple(Tuple):
  text='('
  for i in range(0,len(Tuple)-1):
    entry=express(Tuple[i])
    text=text+entry+', '
  entry=express(Tuple[len(Tuple)-1])
  text=text+entry+')'
  return text


# Prints a matrix as in LaTeX
def latexmatrix(Matrix):
  Matrix=np.array(Matrix)
  text='='+r"\begin{pmatrix} "
  for i in range(0,len(Matrix)):
    for j in range(0,len(Matrix[0])):
      entry=express(Matrix[i][j])
      if j!=len(Matrix[0])-1:
        text=text+entry+' & '
      else:
        if i!=len(Matrix)-1:
          text=text+entry+r"\\ "
        else:
          text=text+entry+' \end{pmatrix}'
  return text


# Quadratic Bezier curve. At least two points, po>1
def quadbez(P0,P1,P2,po): 
  x=[]
  y=[]
  for i in range(0,po):
    t=i/(po-1)
    xt=(1-t)*((1-t)*P0[0]+t*P1[0])+t*((1-t)*P1[0]+t*P2[0])
    yt=(1-t)*((1-t)*P0[1]+t*P1[1])+t*((1-t)*P1[1]+t*P2[1])
    numx=str(xt)
    numx=Decimal(numx).normalize()
    x.append(float(numx))
    numy=str(yt)
    numy=Decimal(numy).normalize()
    y.append(float(numy))
  return (x,y)


# Finds an intermediate point for the Bezier curve. 
def findintermediate(P0,P2,num):
  x=[P0[0],P2[0]]
  y=[P0[1],P2[1]]
  if x[0]<x[1]:
    x0,x1=x
    y0,y1=y
  else:
    x1,x0=x
    y1,y0=y
  r=(((x0-x1)**2+(y0-y1)**2)**0.5)/2
  if num%2==1:
    sign=-1
  else:
    sign=+1
  length=(num+num%2)/30
  d=sign*length
  tangent_vector=Matrix([y0-y1,x1-x0])
  midpoint=Matrix([(x0+x1)/2,(y0+y1)/2])
  intermediate=np.array(midpoint+d*tangent_vector)
  P1=[]
  P1.append(intermediate[0][0])
  P1.append(intermediate[1][0])
  return P1

# Appends a set in a LaTeX document with no more than L elements per line
def setdoc(Set, name, L, doc):
	n=len(Set)
	doc.append(NoEscape(r'\begin{equation*}'))
	doc.append(NoEscape(r'	\begin{split}'))
	r=n%L
	t=int((n-r)/L)
	for i in range(0,t):
		if i==0:
			text='		'+name+'=\{ & '
		else:
  			text='		&'
		for j in range(0,L-1):
  			text=text+str(Set[L*i+j])+', '
		if i!=t-1:
  			text=text+str(Set[L*i+21])+', \\\\ '
		else:
  			if r!=0:
  				text=text+str(Set[L*i+L-1])+', \\\\ '
  			else:
  				text=text+str(Set[L*i+L-1])+' \} '
		doc.append(NoEscape(text))
	if t==0:
		text='		'+name+'=\{ & '
	if r!=0:
  		if t!=0:
  			text='		&'
	  	for j in range(0,r-1):
	  		text=text+str(Set[L*t+j])+', '
	  	text=text+str(Set[L*t+r-1])+'\}'
  		doc.append(NoEscape(text))
	doc.append(NoEscape(r'	\end{split}'))
	doc.append(NoEscape(r'\end{equation*}'))

# Appends a matrix in a LaTeX document with no more than L elements per line
def matrixdoc(matrix, name, L, doc):
	matrix=np.array(matrix)
	n=len(matrix)
	m=len(matrix[0])
	doc.append(NoEscape(r'\begin{equation*}'))
	doc.append(NoEscape(name+'='))
	doc.append(NoEscape(r'	\begin{pmatrix}'))
	#r=n%L
	#t=int((n-r)/L)
	for i in range(0,n):
  		text='		'
  		for j in range(0,m-1):
  			text=text+express(matrix[i][j])+'&'
  		if i!=n-1:
  			text=text+express(matrix[i][m-1])+' \\\\ '
  		else:
  			text=text+express(matrix[i][m-1])
  		doc.append(NoEscape(text))
	doc.append(NoEscape(r'	\end{pmatrix}'))
	doc.append(NoEscape(r'\end{equation*}'))

In [ ]:
# Creates an interactive diagram of the complex showing information about the Voltage, Current, Resitance of each branch and the Potential of each node. Returns the positions of each node, in case we want to create a figure of the maximal forest
def plotcomplexinf(N,B,d,R,V,I,P,positio):
  # Creates graph G, add nodes and branches, assign positions
  G = nx.MultiDiGraph() # We use directed multigraphs
  if positio==0:
	  position=[]
	  for node in N:
	    position.append([random.uniform(0, 1),random.uniform(0, 1)])
	    G.add_node(node, size = 2, pos = position[node])
  else:
  	position=positio
  	for node in N:
  		G.add_node(node, size = 2, pos = position[node])
  for branch in B:
      starting_node=d[branch][0]
      ending_node=d[branch][1]
      G.add_edge(starting_node, ending_node, weight = 1)

  # Creates figure
  fig = go.Figure(layout=go.Layout(
                  title=' ',
                  titlefont_size=16,
                  showlegend=False,
                  hovermode='closest',
                  margin=dict(b=20,l=5,r=5,t=40),
                  xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                  )
  # Add branches to the figure
  connectednodes=[]
  for branch in B:
    sta_node=d[branch][0]
    end_node=d[branch][1]
    P0=G.nodes[sta_node]['pos']
    P2=G.nodes[end_node]['pos']
    num=connectednodes.count({sta_node,end_node})
    P1=findintermediate(P0,P2,num)
    xx,yy=quadbez(P0,P1,P2,20)
    fig.add_trace(go.Scatter(hoverinfo="text",
                        x=xx, y=yy,
                        line=dict(width=0.5, color='cornflowerblue'),
                        marker=dict(opacity=0))
                        )
    y_upper=[]
    y_lower=[]
    for i in range(0,len(yy)):
      y_upper.append(yy[i]+0.0085)
      y_lower.append(yy[i]-0.0085)
    y_lower=y_lower[::-1]
    x_rev=xx[::-1]
    Text='Branch '+str(branch)+': \n '+str(sta_node)+'→'+str(end_node)+', \n Resistance '+express(np.array(R)[branch][branch])+' Ω, \n Current '+express(np.array(I)[branch][0])+' A, \n Voltage '+express(np.array(V)[branch][0]) +' V'
    fig.add_trace(go.Scatter(text=Text,
                      x=xx+x_rev,
                      y=y_upper+y_lower,
                      fill='toself',
                      fillcolor='rgba(0,0,0,0)',
                      line_color='rgba(0,0,0,0)',
                      hoverinfo='text',
                      name='Premium',
                      showlegend=False)
                      )
    connectednodes.append({sta_node,end_node})
  # Add nodes and their information to the figure
  node_x=[]
  node_y=[]
  for node in N:
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)
  node_potential=[]
  node_text=[]
  for node in N:
    node_potential.append(float(P[node]))
    node_text.append('Node '+str(node)+':\n Potential: '+express(P[node])+ ' V')
  fig.add_trace(go.Scatter(text=node_text,
                x=node_x, y=node_y,
                mode='markers',
                hoverinfo='text',
                marker=dict(
                    showscale=True,
                    colorscale='YlGnBu',
                    reversescale=True,
                    color=node_potential,
                    size=10,
                    colorbar=dict(
                      thickness=15,
                      title='Electrostatic potential',
                      xanchor='left',
                      titleside='right'),
                    line_width=2))
                )
  #fig.show()
  return fig, position

In [ ]:
# Creates a diagram of the complex. If we want to plot a maximal forest, we have to insert (N,Bt,d,position), so it can be recognized as a forest: len(Bt)!=len(d)
def plotcomplex(N,B,d,positio):
  # Creates graph G, add nodes and branches, assign positions
  G = nx.MultiDiGraph()
  # If the position is not give, that is if position==0, we create it
  if positio==0:
    position=[]
    for node in N:
      position.append([random.uniform(0, 1),random.uniform(0, 1)])
  else:
    position=positio
  for node in N:
    G.add_node(node, size = 2, pos = position[node])
  for branch in B:
      starting_node=d[branch][0]
      ending_node=d[branch][1]
      G.add_edge(starting_node, ending_node, weight = 1)
  if len(B)==len(d):
    Title='Graph of the one-dimensional complex.'
  else:
    if len(B)+1==len(N):
      Title='Graph of the maximal tree.'
    else:
      numcom=len(N)-len(B)
      Title='Graph of the maximal forest. The complex has '+str(numcom)+' connected components.'
  # Creates figure
  fig = go.Figure(layout=go.Layout(
                  title=' ',
                  titlefont_size=16,
                  showlegend=False,
                  hovermode='closest',
                  margin=dict(b=20,l=5,r=5,t=40),
                  xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                  )
  # Add branches to the figure
  connectednodes=[]
  for branch in B:
    sta_node=d[branch][0]
    end_node=d[branch][1]
    P0=G.nodes[sta_node]['pos']
    P2=G.nodes[end_node]['pos']
    num=connectednodes.count({sta_node,end_node})
    P1=findintermediate(P0,P2,num)
    xx,yy=quadbez(P0,P1,P2,20)
    fig.add_trace(go.Scatter(hoverinfo="text",
                        x=xx, y=yy,
                        line=dict(width=0.5, color='cornflowerblue'),
                        marker=dict(opacity=0))
                        )
    y_upper=[]
    y_lower=[]
    for i in range(0,len(yy)):
      y_upper.append(yy[i]+0.0085)
      y_lower.append(yy[i]-0.0085)
    y_lower=y_lower[::-1]
    x_rev=xx[::-1]
    Text='Branch '+str(branch)+': \n '+str(sta_node)+'→'+str(end_node)
    fig.add_trace(go.Scatter(text=Text,
                      x=xx+x_rev,
                      y=y_upper+y_lower,
                      fill='toself',
                      fillcolor='rgba(0,0,0,0)',
                      line_color='rgba(0,0,0,0)',
                      hoverinfo='text',
                      name='Premium',
                      showlegend=False)
                      )
    connectednodes.append({sta_node,end_node})
  # Add nodes to the figure
  node_x=[]
  node_y=[]
  for node in G.nodes():
    x,y=G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)
  node_text = []
  for node in N:
    node_text.append('Node '+str(node))
  fig.add_trace(go.Scatter(text=node_text,
                x=node_x, y=node_y,
                mode='markers',
                hoverinfo='text',
                marker=dict(
                    color='cornflowerblue',
                    size=10))
                )
  #fig.show()
  if positio==0:
    return fig, Title, position
  else:
  	return fig, Title, position

In [ ]:
# Generates a resistive network of n nodes and b branches, solves it and plotes it
def ResistiveNetwork(n,b):
  # Creates a new folder for all the documents that will be created
  if not os.path.exists("ResNetDoc"):
  	os.mkdir("ResNetDoc")
  # Makes the complex
  (N,B,d)=complex(n,b)
  fig1,title1,position=plotcomplex(N,B,d,0)
  fig1.write_image("ResNetDoc/fig1.png")
  fig1.write_html("ResNetDoc/Graph-of-the-complex.html")
  # Makes a maximal forest
  (Nt,Bt,dt)=maxforest(N,B,d)
  fig2,title2,position=plotcomplex(N,Bt,d,position)
  fig2.write_image("ResNetDoc/fig2.png")
  fig2.write_html("ResNetDoc/Graph-of-the-max-forest.html")
  # Sigma function
  sigma=sigmafu(N,B,d,Bt)
  # Defines the resistors
  (K,W,R)=resistive(len(B))
  # Solves the system of equations 
  (sRsigma,invsRs,answer)=isinvert(R,sigma)
  (I,V)=MeshCurrentMethod(K,W,R,sigma,invsRs)
  # Finds potential
  P,Pvector=potential(N,B,d,V)
  answer=isitapotencial(N,B,d,V,P)
  if answer=='yes':
  	fig3,position=plotcomplexinf(N,B,d,R,V,I,P,position)
  	fig3.write_image("ResNetDoc/fig3.png")
  	fig3.write_html("ResNetDoc/Interactive-graph.html")


  # Makes the LaTeX pdf
  if __name__=='__main__':
  	doc = Document()
  	doc.preamble.append(Command('title', 'Solution of a Resistive Network using the Mesh Current Method'))
  	doc.preamble.append(Command('date', NoEscape(r'\today')))
  	doc.append(NoEscape(r'\maketitle'))
  	with doc.create(Section('One-dimensional Complex')):
  		doc.append('We generated randomly a one-dimensional complex (or graph) with '+str(n)+' nodes and '+str(b)+' branches.')
  	doc.append(NoEscape(r'This one-dimensional complex $\mathcal{G}$ is the triple $(N,B,\varphi)$, generated by just running "ResistiveNetwork(n,b)", with n='+str(n)+' and b='+str(b)+'. The only requisites for this program to work are the following:'))
  	doc.append(Math(data='n>1, b>n-2.'))
  	doc.append('The set of nodes is ')
  	setdoc(N, 'N', 22, doc)
  	doc.append('The set of branches is \n')
  	setdoc(B,'B',22,doc)
  	if b<6:
  		doc.append(NoEscape(r'The function $\varphi\colon B\to N^2$ is defined by'))
  		text=''
  		for branch in range(0,b-1):
  			text=text+r'\varphi('+str(branch)+')='+str(d[branch]) +', \ '
  		text=text+r'\varphi('+str(b-1)+')='+str(d[b-1])+'.'
  		doc.append(NoEscape(r'$$ '+text+'$$'))
  		doc.append(NoEscape(r'For each branch $\beta$, $\varphi(b)=(S_\beta,E_\beta)$ where $S_\beta$ is the starting node of $\beta$ and $E_\beta$ is its ending node.'))
  	if b>5:
  		doc.append(NoEscape(r'The function $\varphi\colon B\to N^2$ is represented by the following table. For each branch $\beta$, $\varphi(\beta)=(S_\beta,E_\beta)$ where $S_\beta$ is the starting node of $\beta$ and $E_\beta$ is its ending node.'))
  		doc.append(NoEscape('\\\\'))
  		doc.append(NoEscape('\\\\'))
  		with doc.create(Tabular('|c|c|c|c|c|c|c|')) as table:
  			table.add_hline()
  			table.add_row(NoEscape('$n$'),NoEscape(r'$\varphi(6n)$'),NoEscape(r'$\varphi(6n+1)$'),NoEscape(r'$\varphi(6n+2)$'),NoEscape(r'$\varphi(6n+3)$'),NoEscape(r'$\varphi(6n+4)$'),NoEscape(r'$\varphi(6n+5)$'))
  			table.add_hline()
  			r=b%6 # We want to write b=6t+r
  			t=int((b-r)/6)
  			for i in range(0,t):
  				table.add_row(str(i),NoEscape(r'$'+str(d[6*i])+'$'),NoEscape(r'$'+str(d[6*i+1])+'$'),NoEscape(r'$'+str(d[6*i+2])+'$'),NoEscape(r'$'+str(d[6*i+3])+'$'),NoEscape(r'$'+str(d[6*i+4])+'$'),NoEscape(r'$'+str(d[6*i+5])+'$'))
  				table.add_hline()
  			if r==1:
  				table.add_row(str(t),NoEscape(r'$'+str(d[6*t])+'$'),' ',' ',' ',' ',' ')
  				table.add_hline()
  			if r==2:
  				table.add_row(str(t),NoEscape(r'$'+str(d[6*t])+'$'),NoEscape(r'$'+str(d[6*t+1])+'$'),' ',' ',' ',' ')
  				table.add_hline()
  			if r==3:
  				table.add_row(str(t),NoEscape(r'$'+str(d[6*t])+'$'),NoEscape(r'$'+str(d[6*t+1])+'$'),NoEscape(r'$'+str(d[6*t+2])+'$'),' ',' ',' ')
  				table.add_hline()
  			if r==4:
  				table.add_row(str(t),NoEscape(r'$'+str(d[6*t])+'$'),NoEscape(r'$'+str(d[6*t+1])+'$'),NoEscape(r'$'+str(d[6*t+2])+'$'),NoEscape(r'$'+str(d[6*t+3])+'$'),' ',' ')
  				table.add_hline()
  			if r==5:
  				table.add_row(str(t),NoEscape(r'$'+str(d[6*t])+'$'),NoEscape(r'$'+str(d[6*t+1])+'$'),NoEscape(r'$'+str(d[6*t+2])+'$'),NoEscape(r'$'+str(d[6*t+3])+'$'),NoEscape(r'$'+str(d[6*t+4])+'$'),' ')
  				table.add_hline()
  	with doc.create(Figure(position='h!')) as graph_complex:
  		graph_complex.add_image("fig1.png")
  		graph_complex.add_caption(title1)
  	#doc.append(NoEscape(r'$(\varphi(0),\ldots,\varphi('+str(b-1)+'))='+latextuple(d)+'$'))
  	with doc.create(Section(NoEscape(r'Maximal forest and $\sigma$ function'))):
  		doc.append(NoEscape(r'We create a complex $\mathcal{T}=(N,B_T,\varphi_T)$ that is a maximal forest of $\mathcal{G}$. If $\mathcal{G}$ is connected, then $\mathcal{T}$ is a maximal tree.'))
  	doc.append('The set of branches of this complex is')
  	setdoc(Bt,'B_T',22,doc)
  	doc.append(NoEscape(r'The function $\varphi_T\colon B_T\to N^2$ behaves exactly as $\varphi$.'))
  	with doc.create(Figure(position='h!')) as graph_forest:
  		graph_forest.add_image("fig2.png")
  		graph_forest.add_caption(title2)
  	doc.append(NoEscape(r'Recall there exists the space of zero-chains $C_0$, which is a real vector space of dimension '+str(n)+', the number of nodes. And the space of one-chains $C_1$ is a real vector space of dimension '+str(b)+', the number of branches.'))
  	doc.append(NoEscape(r'\\ The boundary map $\partial\colon C_1\to C_0$ is defined as $\partial \beta=E_\beta-S_\beta$ for every branch $\beta\in B$. We denote by $Z_1$ the kernel of $\partial$ and by $B_0$ the image of it. '))
  	doc.append(NoEscape(r' If the complex does not have meshes, then $Z_1=\{0\}$, if it does have meshes, then $Z_1$ has a basis consisting only of meshes. \\\\'))
  	doc.append(NoEscape(r'The first homology group of the complex $\mathcal{G}$ is defined just as $H_1=Z_1/\{0\}$, thus it is isomorphic to $Z_1$. Via this isomorphism, $H_1$ is said to be have a basis consisting of meshes. '))
  	doc.append(NoEscape(r' The inclusion map $\sigma\colon H_1\to C_1$ which identifies $H_1$ with the subspace $Z_1$ is physically interpreted to convert currents on meshes to currents on the branches.'))
  	doc.append(NoEscape('\\\\'))
  	if invsRs==0:
  		doc.append(NoEscape(r"In our case, the complex doesn't have meshes, so $Z_1$ is trivial, such as $\sigma$"))
  	else:
  		doc.append(NoEscape(r'In our case, we have a basis of $Z_1$ consisting of '+str(len(B)-len(Bt))+' meshes and the following matrix representation of the $\sigma$ function (relative to the basis of meshes in $H_1$ and the basis of branches in $C_1$)'))
  		matrixdoc(sigma, '\sigma', 6, doc)
  	with doc.create(Section('Simulating resistors')):
  		doc.append(NoEscape(r'\setcounter{MaxMatrixCols}{20}'))
  		doc.append(NoEscape(r'In this section, we simulate randomly the properties of the '+str(b)+r" resistors.  For each resistor $\alpha$, we have the quantities $r_\alpha, K_\alpha$ and $V^\alpha$, such that the Ohm's law follows: "))
  		doc.append(NoEscape(r'$$V^\alpha-W^\alpha=r_\alpha(I_\alpha-K_\alpha).$$'))
  		doc.append(NoEscape(r'Were $V^\alpha$ is the voltage on $\alpha$ and $I_\alpha$ is the current on $\alpha$. If we let  $\mathbf{K}=[K_0,\ldots,K_{'+str(b-1)+'}]^T$ be the matrix representation of a transformation in $C_1$, $\mathbf{V}=[V^0,\ldots,V^{'+str(b-1)+'}]^T$ be the matrix representation of a transformation in $C^1$ and '))
  		if b==1:
  			doc.append(NoEscape(r'$R\colon C_1\to C^1$ be represented by [r_0].'))
  		else:
  			doc.append(NoEscape(r'$R\colon C_1\to C^1$ be represented by the following matrix'))
  			doc.append(NoEscape(r'\begin{equation*}'))
  			doc.append('R=')
  			doc.append(NoEscape(r'\begin{pmatrix}'))
  			doc.append(NoEscape(r'	r_{0} & 0 & \cdots & 0 \\'))
  			doc.append(NoEscape(r'	0 & r_{1} & \cdots & 0 \\'))
  			doc.append(NoEscape(r'	\vdots & \vdots & \ddots & \vdots \\'))
  			doc.append(NoEscape(r'	0 & 0 & \cdots & r_{'+str(b-1)+'}'))
  			doc.append(NoEscape(r'\end{pmatrix}'))
  			doc.append(NoEscape(r'\end{equation*}'))
  		doc.append("Then, Ohm's law amounts to")
  		doc.append(NoEscape(r'$$\mathbf{V}-\mathbf{W}=R(\mathbf{I}-\mathbf{K}),$$'))
  		doc.append(NoEscape(r'where $\mathbf{V}\in C^1$ represent the voltages and $\mathbf{I}\in C_1$ the currents.'))
  		doc.append(NoEscape('\\\\'))
  		doc.append(NoEscape(r'We randomly generated the parameters of each resistor so that we have a linear resistive network (meaning $r_\alpha>0$ for each $\alpha$). We obtained the following'))
  		matrixdoc(R, 'R', 6, doc)
  		matrixdoc(K, '\mathbf{K}', 6, doc)
  		matrixdoc(W, '\mathbf{W}', 6, doc)
  	with doc.create(Section('Solving the system of equations')):
  		doc.append('The system of equations consists of the following')
  		doc.append(NoEscape(r'\begin{itemize}'))
  		doc.append(NoEscape(r"\item Ohm's law: $\mathbf{V}-\mathbf{W}=R(\mathbf{I}-\mathbf{K}).$"))
  		doc.append(NoEscape(r"\item Kirchhoff's current law: $\partial \mathbf{I}=\mathbf{0}.$"))
  		doc.append(NoEscape(r"\item Kirchhoff's voltage law: There exists a $\phi\in C^0$ such that $\mathbf{V}=-d\phi$."))
  		doc.append(NoEscape(r'\end{itemize}'))
  		if invsRs==0:
  			doc.append(NoEscape(r"Since there are no meshes in the complex, $Z_1$ is trivial and if $\mathbf{I}\in C_1$ satisfies Kirchhoff's current law, then $\mathbf{I}=\mathbf{0}=[0,\ldots,0]^T$."))
  			doc.append(NoEscape(r"On the other hand, if $\mathbf{V}$ satisfies Ohm's Law, we just simply have $\mathbf{V}=\mathbf{W}+R(\mathbf{I}-\mathbf{K})$, that is"))
  			matrixdoc(V,'\mathbf{V}',6,doc)
  		else:
  			doc.append(NoEscape(r"If $\mathbf{I}$ satisfies Kirchhoff's current law, then $\mathbf{I}\in Z_1$, thus $\mathbf{I}$ is in the image of $\sigma$, meaning there exists a $\mathbf{J}\in H_1$ such that $\sigma\mathbf{J}=\mathbf{I}.$ "))
  			doc.append(NoEscape(r"  Kirchhoff's voltage law implies that $s\mathbf{V}=\mathbf{0}$, where $s\colon C^1\to H^1$ is the adjoint map of $\sigma$, so we calculate"))
  			doc.append(NoEscape(r"$$(sR\sigma)\mathbf{J}=s(R\mathbf{K}-\mathbf{W}).$$"))
  			doc.append(NoEscape(r"Then, if $sR\sigma$ is invertible, we can find $\mathbf{J}$ and $\mathbf{I}=\sigma\mathbf{J}$. Using Python, we calculated the inverse of $sR\sigma$ and obtained"))
  			matrixdoc(invsRs,'(sR\sigma)^{-1}',6,doc)
  			doc.append('From which we can obtain')
  			matrixdoc(I,'\mathbf{I}',6,doc)
  			doc.append(NoEscape(r"Finally, if $\mathbf{V}$ satisfies Ohm's Law, we just simply have $\mathbf{V}=\mathbf{W}+R(\mathbf{I}-\mathbf{K})$, that is"))
  			matrixdoc(V,'\mathbf{V}',6,doc)
  			doc.append(NoEscape(r"By construction, $\mathbf{I}$ and $\mathbf{V}$ satisfy Ohm's law and Kirchhoff's current law, but it is not obvious that Kirchhoff's voltage law holds as we stated it. Using Python, we created a function $P$ that must be a potential function if it exists."))
  			matrixdoc(Pvector,'P',6,doc)
  			doc.append(NoEscape(r"The branch 0 goes from 0 to 1, and note that $$(-dP)(0)=-(P(1)-P(0))="+express(P[0])+'-('+express(P[1])+')='+express(V[0])+'=V(0),$$ as expected. '))
  			if answer=='yes':
  				doc.append(NoEscape(r"Using Python, we verified that $V(\alpha)=(-dP)(\alpha)$ for each branch $\alpha$, so Kirchhoff's voltage law is satisfied."))
  				with doc.create(Figure(position='h!')) as int_graph:
  					int_graph.add_image("fig3.png")
  					int_graph.add_caption("Find an interactive graph of the complex with its solutions at `ResNetDoc/Interactive-graph.html'.")
  			if answer=='no': # Theorically, this never happens
  				doc.append(r"Using Python, we refuted that $V(\alpha)=(-dP)(\alpha)$ for each branch $\alpha$, so Kirchhoff's voltage law is not satisfied.")

  	doc.generate_pdf("ResNetDoc/ResistiveNetwork", clean_tex=False)

In [ ]:
# Solves a given electrical network (N,B,d) with resistors defined by the matrices K, W, R. If there are no given positions for each node, enter position=0
def Solve(N,B,d,K,W,R,position):
  n=len(N)
  b=len(B)
  # Creates a new folder for all the documents that will be created
  if not os.path.exists("SolutionComplex"):
    os.mkdir("SolutionComplex")
  # Plots complex
  fig1,title1,position=plotcomplex(N,B,d,position)
  fig1.write_image("SolutionComplex/fig1.png")
  fig1.write_html("SolutionComplex/Graph-of-the-complex.html")
  # Finds a maximal forest
  (Nt,Bt,dt)=maxforest(N,B,d)
  fig2,title2,position=plotcomplex(N,Bt,d,position)
  fig2.write_image("SolutionComplex/fig2.png")
  fig2.write_html("SolutionComplex/Graph-of-the-max-forest.html")
  # Sigma function
  sigma=sigmafu(N,B,d,Bt)
  # Solves the system of equations 
  (sRsigma,invsRs,answer)=isinvert(R,sigma)
  if answer==1:
    (I,V)=MeshCurrentMethod(K,W,R,sigma,invsRs)
    # Finds potential
    P,Pvector=potential(N,B,d,V)
    answer2=isitapotencial(N,B,d,V,P)
    fig3,position=plotcomplexinf(N,B,d,R,V,I,P,position)
    fig3.write_image("SolutionComplex/fig3.png")
    fig3.write_html("SolutionComplex/Interactive-graph.html")


  # Makes the LaTeX pdf
  if __name__=='__main__':
    doc = Document()
    doc.preamble.append(Command('title', 'Solution of an Electrical Network using the Mesh Current Method'))
    doc.preamble.append(Command('date', NoEscape(r'\today')))
    doc.append(NoEscape(r'\maketitle'))
    with doc.create(Section('One-dimensional Complex')):
      doc.append(NoEscape(r'We work with the one-dimensional complex $\mathcal{G}=(N,B,\varphi)$. Let $n=|N|$ be the number of nodes and $b=|B|$ the number of branches. By the definition of one-dimensional complex, we have the following inequalities'))
    doc.append(Math(data='n>1, b>n-2.'))
    doc.append('The set of nodes is \n')
    setdoc(N, 'N', 22, doc)
    doc.append('and the set of branches is \n')
    setdoc(B,'B',22,doc)
    if b<6:
      doc.append(NoEscape(r'The function $\varphi\colon B\to N^2$ is defined by'))
      text=''
      for branch in range(0,b-1):
        text=text+r'\varphi('+str(branch)+')='+str(d[branch]) +', \ '
      text=text+r'\varphi('+str(b-1)+')='+str(d[b-1])+'.'
      doc.append(NoEscape(r'$$ '+text+'$$'))
      doc.append(NoEscape(r'For each branch $\beta$, $\varphi(b)=(S_\beta,E_\beta)$ where $S_\beta$ is the starting node of $\beta$ and $E_\beta$ is its ending node.'))
    if b>5:
      doc.append(NoEscape(r'The function $\varphi\colon B\to N^2$ is represented by the following table. For each branch $\beta$, $\varphi(\beta)=(S_\beta,E_\beta)$ where $S_\beta$ is the starting node of $\beta$ and $E_\beta$ is its ending node.'))
      doc.append(NoEscape('\\\\'))
      doc.append(NoEscape('\\\\'))
      with doc.create(Tabular('|c|c|c|c|c|c|c|')) as table:
        table.add_hline()
        table.add_row(NoEscape('$n$'),NoEscape(r'$\varphi(6n)$'),NoEscape(r'$\varphi(6n+1)$'),NoEscape(r'$\varphi(6n+2)$'),NoEscape(r'$\varphi(6n+3)$'),NoEscape(r'$\varphi(6n+4)$'),NoEscape(r'$\varphi(6n+5)$'))
        table.add_hline()
        r=b%6 # We want to write b=6t+r
        t=int((b-r)/6)
        for i in range(0,t):
          table.add_row(str(i),NoEscape(r'$'+str(d[6*i])+'$'),NoEscape(r'$'+str(d[6*i+1])+'$'),NoEscape(r'$'+str(d[6*i+2])+'$'),NoEscape(r'$'+str(d[6*i+3])+'$'),NoEscape(r'$'+str(d[6*i+4])+'$'),NoEscape(r'$'+str(d[6*i+5])+'$'))
          table.add_hline()
        if r==1:
          table.add_row(str(t),NoEscape(r'$'+str(d[6*t])+'$'),' ',' ',' ',' ',' ')
          table.add_hline()
        if r==2:
          table.add_row(str(t),NoEscape(r'$'+str(d[6*t])+'$'),NoEscape(r'$'+str(d[6*t+1])+'$'),' ',' ',' ',' ')
          table.add_hline()
        if r==3:
          table.add_row(str(t),NoEscape(r'$'+str(d[6*t])+'$'),NoEscape(r'$'+str(d[6*t+1])+'$'),NoEscape(r'$'+str(d[6*t+2])+'$'),' ',' ',' ')
          table.add_hline()
        if r==4:
          table.add_row(str(t),NoEscape(r'$'+str(d[6*t])+'$'),NoEscape(r'$'+str(d[6*t+1])+'$'),NoEscape(r'$'+str(d[6*t+2])+'$'),NoEscape(r'$'+str(d[6*t+3])+'$'),' ',' ')
          table.add_hline()
        if r==5:
          table.add_row(str(t),NoEscape(r'$'+str(d[6*t])+'$'),NoEscape(r'$'+str(d[6*t+1])+'$'),NoEscape(r'$'+str(d[6*t+2])+'$'),NoEscape(r'$'+str(d[6*t+3])+'$'),NoEscape(r'$'+str(d[6*t+4])+'$'),' ')
          table.add_hline()
    with doc.create(Figure(position='h!')) as graph_complex:
      graph_complex.add_image("fig1.png")
      graph_complex.add_caption(title1)
    #doc.append(NoEscape(r'$(\varphi(0),\ldots,\varphi('+str(b-1)+'))='+latextuple(d)+'$'))
    with doc.create(Section(NoEscape(r'Maximal forest and $\sigma$ function'))):
      doc.append(NoEscape(r'We create a complex $\mathcal{T}=(N,B_T,\varphi_T)$ that is a maximal forest of $\mathcal{G}$. If $\mathcal{G}$ is connected, then $\mathcal{T}$ is a maximal tree.'))
    doc.append('The set of branches of this complex is')
    setdoc(Bt,'B_T',22,doc)
    doc.append(NoEscape(r'The function $\varphi_T\colon B_T\to N^2$ behaves exactly as $\varphi$.'))
    with doc.create(Figure(position='h!')) as graph_forest:
      graph_forest.add_image("fig2.png")
      graph_forest.add_caption(title2)
    doc.append(NoEscape(r'Recall there exists the space of zero-chains $C_0$, which is a real vector space of dimension '+str(n)+', the number of nodes. And the space of one-chains $C_1$ is a real vector space of dimension '+str(b)+', the number of branches.'))
    doc.append(NoEscape(r'\\ The boundary map $\partial\colon C_1\to C_0$ is defined as $\partial \beta=E_\beta-S_\beta$ for every branch $\beta\in B$. We denote by $Z_1$ the kernel of $\partial$ and by $B_0$ the image of it. '))
    doc.append(NoEscape(r' If the complex does not have meshes, then $Z_1=\{0\}$, if it does have meshes, then $Z_1$ has a basis consisting only of meshes. \\\\'))
    doc.append(NoEscape(r'The first homology group of the complex $\mathcal{G}$ is defined just as $H_1=Z_1/\{0\}$, thus it is isomorphic to $Z_1$. Via this isomorphism, $H_1$ is said to be have a basis consisting of meshes. '))
    doc.append(NoEscape(r' The inclusion map $\sigma\colon H_1\to C_1$ which identifies $H_1$ with the subspace $Z_1$ is physically interpreted to convert currents on meshes to currents on the branches.'))
    doc.append(NoEscape('\\\\'))
    if invsRs==0:
      doc.append(NoEscape(r"In our case, the complex doesn't have meshes, so $Z_1$ is trivial, such as $\sigma$"))
    else:
      doc.append(NoEscape(r'In our case, we have a basis of $Z_1$ consisting of '+str(len(B)-len(Bt))+' meshes and the following matrix representation of the $\sigma$ function (relative to the basis of meshes in $H_1$ and the basis of branches in $C_1$)'))
      matrixdoc(sigma, '\sigma', 6, doc)
    with doc.create(Section('Properties of the resistors')):
      doc.append(NoEscape(r'\setcounter{MaxMatrixCols}{20}'))
      doc.append(NoEscape(r"For each resistor $\alpha$, we have the quantities $r_\alpha, K_\alpha$ and $V^\alpha$, such that the Ohm's law follows: "))
      doc.append(NoEscape(r'$$V^\alpha-W^\alpha=r_\alpha(I_\alpha-K_\alpha).$$'))
      doc.append(NoEscape(r'Were $V^\alpha$ is the voltage on $\alpha$ and $I_\alpha$ is the current on $\alpha$. If we let  $\mathbf{K}=[K_0,\ldots,K_{'+str(b-1)+'}]^T$ be the matrix representation of a transformation in $C_1$, $\mathbf{V}=[V^0,\ldots,V^{'+str(b-1)+'}]^T$ be the matrix representation of a transformation in $C^1$ and '))
      if b==1:
        doc.append(NoEscape(r'$R\colon C_1\to C^1$ be represented by [r_0].'))
      else:
        doc.append(NoEscape(r'$R\colon C_1\to C^1$ be represented by the following matrix'))
        doc.append(NoEscape(r'\begin{equation*}'))
        doc.append('R=')
        doc.append(NoEscape(r'\begin{pmatrix}'))
        doc.append(NoEscape(r'  r_{0} & 0 & \cdots & 0 \\'))
        doc.append(NoEscape(r'  0 & r_{1} & \cdots & 0 \\'))
        doc.append(NoEscape(r'  \vdots & \vdots & \ddots & \vdots \\'))
        doc.append(NoEscape(r'  0 & 0 & \cdots & r_{'+str(b-1)+'}'))
        doc.append(NoEscape(r'\end{pmatrix}'))
        doc.append(NoEscape(r'\end{equation*}'))
      doc.append("Then, Ohm's law amounts to")
      doc.append(NoEscape(r'$$\mathbf{V}-\mathbf{W}=R(\mathbf{I}-\mathbf{K}),$$'))
      doc.append(NoEscape(r'where $\mathbf{V}\in C^1$ represent the voltages and $\mathbf{I}\in C_1$ the currents.'))
      doc.append('\\\\')
      doc.append(NoEscape(r'The vectors that represent the properties of our resistors are the following:'))
      matrixdoc(R, 'R', 6, doc)
      matrixdoc(K, '\mathbf{K}', 6, doc)
      matrixdoc(W, '\mathbf{W}', 6, doc)
    with doc.create(Section('Solving the system of equations')):
      doc.append('Our system of equations consists of the following')
      doc.append(NoEscape(r'\begin{itemize}'))
      doc.append(NoEscape(r"\item Ohm's law: $\mathbf{V}-\mathbf{W}=R(\mathbf{I}-\mathbf{K}).$"))
      doc.append(NoEscape(r"\item Kirchhoff's current law: $\partial \mathbf{I}=\mathbf{0}.$"))
      doc.append(NoEscape(r"\item Kirchhoff's voltage law: There exists a $\phi\in C^0$ such that $\mathbf{V}=-d\phi$."))
      doc.append(NoEscape(r'\end{itemize}'))
      if invsRs==0:
        doc.append(NoEscape(r"Since there are no meshes in the complex, $Z_1$ is trivial and if $\mathbf{I}\in C_1$ satisfies Kirchhoff's current law, then $\mathbf{I}=\mathbf{0}=[0,\ldots,0]^T$."))
        doc.append(NoEscape(r"On the other hand, if $\mathbf{V}$ satisfies Ohm's Law, we just simply have $\mathbf{V}=\mathbf{W}+R(\mathbf{I}-\mathbf{K})$, that is"))
        matrixdoc(V,'\mathbf{V}',6,doc)
      else:
        doc.append(NoEscape(r"If $\mathbf{I}$ satisfies Kirchhoff's current law, then $\mathbf{I}\in Z_1$, thus $\mathbf{I}$ is in the image of $\sigma$, meaning there exists a $\mathbf{J}\in H_1$ such that $\sigma\mathbf{J}=\mathbf{I}.$ "))
        doc.append(NoEscape(r" Kirchhoff's voltage law implies that $s\mathbf{V}=\mathbf{0}$, where $s\colon C^1\to H^1$ is the adjoint map of $\sigma$, so we calculate"))
        doc.append(NoEscape(r"$$(sR\sigma)\mathbf{J}=s(R\mathbf{K}-\mathbf{W}).$$"))
        doc.append(NoEscape(r"Then, if $sR\sigma$ is invertible, we can find $\mathbf{J}$ and $\mathbf{I}=\sigma\mathbf{J}$. Using Python, we calculated the inverse of $sR\sigma$ and obtained"))
        matrixdoc(invsRs,'(sR\sigma)^{-1}',6,doc)
        doc.append('From which we can obtain')
        matrixdoc(I,'\mathbf{I}',6,doc)
        doc.append(NoEscape(r"Finally, if $\mathbf{V}$ satisfies Ohm's Law, we just simply have $\mathbf{V}=\mathbf{W}+R(\mathbf{I}-\mathbf{K})$, that is"))
        matrixdoc(V,'\mathbf{V}',6,doc)
        doc.append(NoEscape(r"By construction, $\mathbf{I}$ and $\mathbf{V}$ satisfy Ohm's law and Kirchhoff's current law, but it is not obvious that Kirchhoff's voltage law holds as we stated it. Using Python, we created a function $P$ that must be a potential function if it exists."))
        matrixdoc(Pvector,'P',6,doc)
        doc.append(NoEscape(r"The branch 0 goes from 0 to 1, and note that $$(-dP)(0)=-(P(1)-P(0))="+express(P[0])+'-('+express(P[1])+')='+express(V[0])+'=V(0),$$ as expected. '))
        if answer2=='yes':
          doc.append(NoEscape(r"Using Python, we verified that $V(\alpha)=(-dP)(\alpha)$ for each branch $\alpha$, so Kirchhoff's voltage law is satisfied."))
          with doc.create(Figure(position='h!')) as int_graph:
            int_graph.add_image("fig3.png")
            int_graph.add_caption("Find an interactive graph of the complex with its solutions at `SolutionComplex/Interactive-graph.html'.")
        if answer2=='no': # Theorically, this never happens
          doc.append(r"Using Python, we refuted that $V(\alpha)=(-dP)(\alpha)$ for each branch $\alpha$, so Kirchhoff's voltage law is not satisfied.")
    doc.generate_pdf("SolutionComplex/Solution", clean_tex=False)

In [ ]:
# Makes an experiment: change 12 and 18 for the number of nodes and branches desired:
ResistiveNetwork(12,18)